# Logging with Multiple Loggers

In large applications, you often need different loggers for different modules or components. Multiple loggers allow you to control logging behavior separately for different parts of your application.

## Why Use Multiple Loggers?

- **Modularity**: Separate logging for different modules (database, API, authentication, etc.)
- **Fine-grained Control**: Different log levels and handlers for different components
- **Better Organization**: Easy to filter and analyze logs by component
- **Flexibility**: Configure each logger independently

## What We'll Learn

1. Creating Named Loggers
2. Logger Hierarchy
3. Logger Propagation
4. Configuring Multiple Loggers
5. Real-World Examples with Multiple Modules

---

## 1. Creating Named Loggers

Use `logging.getLogger(name)` to create or get a logger with a specific name. It's recommended to use `__name__` as the logger name, which automatically uses the module name.

In [ ]:
import logging

# Configure root logger
logging.basicConfig(level=logging.DEBUG, format='%(name)s - %(levelname)s - %(message)s')

# Create different named loggers
logger_db = logging.getLogger('database')
logger_api = logging.getLogger('api')
logger_auth = logging.getLogger('authentication')

# Use the loggers
logger_db.info("Connected to database")
logger_api.warning("API rate limit approaching")
logger_auth.error("Invalid credentials")

# Notice how the logger name appears in each message

---

## 2. Logger Hierarchy

Loggers follow a hierarchical structure using dot notation (`.`). Child loggers inherit configuration from parent loggers.

**Example Hierarchy:**
```
root
├── myapp
│   ├── myapp.database
│   ├── myapp.api
│   └── myapp.auth
└── thirdparty
    └── thirdparty.library
```

The logger `myapp.database` is a child of `myapp`, which is a child of the root logger.

In [ ]:
import logging

# Configure root logger
logging.basicConfig(level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')

# Create parent and child loggers
parent_logger = logging.getLogger('myapp')
child_logger = logging.getLogger('myapp.database')
grandchild_logger = logging.getLogger('myapp.database.connection')

# Set level on parent
parent_logger.setLevel(logging.WARNING)

# Child inherits from parent
parent_logger.info("This won't show (level is WARNING)")
parent_logger.warning("Parent warning message")
child_logger.warning("Child warning message")
grandchild_logger.error("Grandchild error message")

---

## 3. Logger Propagation

By default, log messages propagate up the logger hierarchy. This means a child logger's messages are passed to its parent's handlers.

**Propagation** = `True` (default): Messages go to both child and parent handlers  
**Propagation** = `False`: Messages only go to child's handlers

In [ ]:
import logging

# Create parent and child loggers
parent = logging.getLogger('parent')
child = logging.getLogger('parent.child')

# Add handler to parent
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('%(name)s - %(message)s'))
parent.addHandler(handler)
parent.setLevel(logging.DEBUG)

print("=== With Propagation (default) ===")
child.propagate = True  # Default
child.warning("This message propagates to parent")

print("\n=== Without Propagation ===")
child.propagate = False
child.warning("This message does NOT propagate to parent")

---

## 4. Configuring Multiple Loggers with Different Handlers

Each logger can have its own handlers, formatters, and log levels. This allows fine-grained control over logging behavior.

In [ ]:
import logging

# Create loggers for different modules
db_logger = logging.getLogger('database')
api_logger = logging.getLogger('api')

# Configure database logger
db_handler = logging.FileHandler('database.log')
db_handler.setFormatter(logging.Formatter('%(asctime)s - DB - %(levelname)s - %(message)s'))
db_logger.addHandler(db_handler)
db_logger.setLevel(logging.DEBUG)
db_logger.propagate = False  # Don't propagate to root

# Configure API logger
api_handler = logging.FileHandler('api.log')
api_handler.setFormatter(logging.Formatter('%(asctime)s - API - %(levelname)s - %(message)s'))
api_logger.addHandler(api_handler)
api_logger.setLevel(logging.INFO)
api_logger.propagate = False

# Use the loggers
db_logger.debug("Database query executed")
db_logger.info("Connection pool initialized")

api_logger.info("API request received")
api_logger.warning("Rate limit exceeded")

print("Check database.log and api.log files!")

---

## 5. Practical Example: Multi-Module Application

Simulating a real application with separate modules, each with its own logger.

In [ ]:
import logging

# Setup console handler for all loggers
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_format = logging.Formatter('%(name)-20s - %(levelname)-8s - %(message)s')
console_handler.setFormatter(console_format)

# Database Module Logger
class DatabaseModule:
    def __init__(self):
        self.logger = logging.getLogger('app.database')
        self.logger.setLevel(logging.DEBUG)
        self.logger.addHandler(console_handler)
        self.logger.propagate = False
    
    def connect(self):
        self.logger.info("Connecting to database...")
        self.logger.debug("Connection parameters loaded")
    
    def query(self, sql):
        self.logger.debug(f"Executing query: {sql}")
        self.logger.info("Query executed successfully")

# API Module Logger
class APIModule:
    def __init__(self):
        self.logger = logging.getLogger('app.api')
        self.logger.setLevel(logging.INFO)
        self.logger.addHandler(console_handler)
        self.logger.propagate = False
    
    def handle_request(self, endpoint):
        self.logger.info(f"Received request: {endpoint}")
        self.logger.debug("Request validation passed")  # Won't show (level is INFO)

# Authentication Module Logger
class AuthModule:
    def __init__(self):
        self.logger = logging.getLogger('app.auth')
        self.logger.setLevel(logging.WARNING)
        self.logger.addHandler(console_handler)
        self.logger.propagate = False
    
    def login(self, username):
        self.logger.info(f"Login attempt: {username}")  # Won't show (level is WARNING)
        self.logger.warning("Multiple failed login attempts detected")

# Test the multi-logger system
db = DatabaseModule()
api = APIModule()
auth = AuthModule()

print("=== Application Start ===\n")
db.connect()
db.query("SELECT * FROM users")
print()
api.handle_request("/api/users")
print()
auth.login("admin")
print("\n=== Application End ===")

---

## Summary

**Key Takeaways:**

1. **Named Loggers**: Create loggers with `logging.getLogger(name)`
2. **Hierarchy**: Use dot notation for parent-child relationships
3. **Propagation**: Control whether messages flow up the hierarchy
4. **Independent Configuration**: Each logger can have its own:
   - Log level
   - Handlers
   - Formatters
   - Propagation settings
5. **Best Practices**:
   - Use `__name__` for module-level loggers
   - Create a logger per module/component
   - Set `propagate=False` to avoid duplicate messages
   - Configure parent loggers for common settings
   - Use hierarchy for organizational structure

**Benefits of Multiple Loggers:**
- Better code organization
- Easier debugging (filter by module)
- Flexible configuration per component
- Scalable for large applications